In [1]:
import squidpy as sq
import scanpy as sc
import pandas as pd 
import numpy as np 
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from adipo_finder import utils as seg_utils
from adipo_finder import segmentation as seg
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import adipo_finder.new_adipofinder_code as nac
from skimage import measure, morphology, segmentation, feature
from scipy.ndimage import distance_transform_edt, label, binary_dilation
from sklearn.datasets import make_blobs
import torch
import pickle


/home/jgustafs/miniconda3/envs/adipo_env/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/jgustafs/miniconda3/envs/adipo_env/lib/python3.10/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
adata = sc.read('../../../data/spe_final.h5ad')

In [3]:
all_ids=adata.obs['library_id'].unique()

In [4]:
#get ground truth ids
import os
import glob
base_path = "../../../data/Manual_adipocyte_segmentation/"

# find all files ending with " step 1.tif"
files = glob.glob(os.path.join(base_path, "* step 3.tif"))

# extract IDs (everything before " step 1.tif")
gt_ids = [os.path.basename(f).replace(" step 3.tif", "") for f in files]

print(gt_ids)

['ROI001_02454_ROI_1', 'ROI001_026859_ROI_1', 'ROI001_08147_ROI_1', 'ROI001_31479_ROI_1', 'ROI001_40423_ROI_2', 'ROI001_44243_ROI_1', 'ROI001_50286_ROI_1', 'ROI001_62626_ROI_1', 'ROI001_70736_ROI_1', 'ROI001_J33533_ROI_1', 'ROI002_40900_ROI_1', 'ROI002_45504_ROI_1', 'ROI002_74014_ROI_1', 'ROI003_37912_ROI_1', 'ROI004_55218_ROI_1']


In [5]:
print(f"tot samples: {len(all_ids)}, ground truth samples: {len(gt_ids)}")

tot samples: 60, ground truth samples: 15


In [6]:
from PIL import Image
input_images = []
ground_truth_images = []
for id in all_ids:
    input_images.append(seg_utils.Preprocessing.extract_segmentation_image(adata, id))
for id in gt_ids:
    ground_truth_images.append(np.array(Image.open(base_path + id + " step 4.png")))

In [7]:
#this takes a while to run
full_df, all_bin_mask, all_distance, all_segmented, all_cleaned_mask = nac.prepare_datasets(all_ids, gt_ids, input_images, ground_truth_images, min_distance_seg_init = 20, min_island_area = 50)

Processing ROI001_50286_ROI_1
Processing ROI002_59748_ROI_1
Processing ROI001_60004_ROI_1
Processing ROI001_08147_ROI_1
Processing ROI002_11952_ROI_1
Processing ROI002_63929_ROI_1
Processing ROI001_49270_ROI_1
Processing ROI002_34234_ROI_2
Processing ROI002_08547_ROI_001
Processing ROI003_34416_ROI_3
Processing ROI001_40423_ROI_2
Processing ROI001_J33533_ROI_1
Processing ROI002_45504_ROI_1
Processing ROI001_58447_ROI_1
Processing ROI001_29469_ROI_1
Processing ROI001_47690_ROI_1
Processing ROI001_57524_ROI_1
Processing ROI003_37912_ROI_1
Processing ROI001_79790_ROI_1
Processing ROI001_76310_ROI_001
Processing ROI002_62914_ROI_1
Processing ROI001_44337_ROI_1
Processing ROI001_04011_ROI_1
Processing ROI002_44149_ROI_1
Processing ROI001_26861_ROI_1
Processing ROI002_74006_ROI_1
Processing ROI001_29095_ROI_1
Processing ROI001_55153_ROI_1
Processing ROI006_44029_ROI_1
Processing ROI001_62626_ROI_1
Processing ROI002_20945_ROI_1
Processing ROI001_65731_ROI_1
Processing ROI002_40900_ROI_1
Proce

In [8]:
full_df

,segment_id,area,eccentricity,compactness,max_dist,distance_from_shore,frac_ring_other_objects,frac_ring_tissue,ground_truth,image_id
0,1,57.628118,0.466822,0.582712,36.069378,27.0,0.613911,0.316182,0.0,ROI001_50286_ROI_1
1,2,20.760539,0.815154,0.775102,15.000000,31.0,0.742250,0.251550,0.0,ROI001_50286_ROI_1
2,3,7.483315,0.266059,1.029712,4.123106,105.0,0.264184,0.519504,0.0,ROI001_50286_ROI_1
3,4,34.698703,0.519269,0.609621,14.560220,72.0,0.121420,0.762314,0.0,ROI001_50286_ROI_1
4,5,27.000000,0.861668,0.633854,21.260292,36.0,0.730692,0.261249,0.0,ROI001_50286_ROI_1
...,...,...,...,...,...,...,...,...,...,...
77483,713,30.545049,0.739310,0.659864,12.000000,136.0,0.000651,0.901693,NaN,ROI001_73506_ROI_1
77484,714,18.654758,0.597981,0.831363,11.401754,129.0,0.423116,0.558794,NaN,ROI001_73506_ROI_1
77485,715,67.394362,0.489088,0.641827,70.178344,7.0,0.874922,0.113594,NaN,ROI001_73506_ROI_1
77486,716,28.301943,0.903215,0.634253,14.000000,92.0,0.326693,0.646746,NaN,ROI001_73506_ROI_1


In [9]:
full_df.to_csv("../data/exported_feature_data.txt", sep='\t', index=False)
